# 实验1

In [1]:
def Check(text, projection):
    '这个函数用于检查输入的字符是否合法、输入的数据是否在范围内，如果不合法返回False，合法返回True'
    inputNum = text.split(',')
    if len(text.split(',')) != 2:
        return False
    for i in range(2):
        try:
            inputNum[i] = float(inputNum[i])
        except:
            return False
    if projection == "WGS84":
        if inputNum[0] < -180 or inputNum[0] > 180:
            return False
        elif inputNum[1] <= -90 or inputNum[1] >= 90:
            return False
    elif projection == "WebMercator":
        if inputNum[0] < -20037508.34 or inputNum[0] > 20037508.34:
            return False
    return True


print("------以下是WGS84转Web墨卡托------")
from math import *

textWGS = input("输入经度和纬度坐标（之间用逗号分隔），输入完成后请按回车键：")
while Check(textWGS, "WGS84") == False:
    textWGS = input("输入错误！请以正确的格式输入经度和纬度坐标（之间用逗号分隔）：")
lon = float(textWGS.split(',')[0])
lat = float(textWGS.split(',')[1])
r = 20037508.34
x = lon / 180 * r
#如果是负数
y = log(tan(pi / 4 + lat * pi / 360)) * r / pi
print("x:{:.2f}, y:{:.2f}".format(x, y))

print("------以下是Web墨卡托转WGS84------")
textWeb = input("输入x和y投影坐标（之间用逗号分隔），输入完成后请按回车键：")
while Check(textWeb, "WebMercator") == False:
    textWeb = input("输入错误！请以正确的格式输入x和y投影坐标（之间用逗号分隔）：")
X = float(textWeb.split(',')[0])
Y = float(textWeb.split(',')[1])
r = 20037508.34
Lon = X / r * 180
Lat = atan(e ** (Y * pi / r)) * 360 / pi - 90
print("经度:{:.2f}, 纬度{:.2f}".format(Lon, Lat))

------以下是WGS84转Web墨卡托------
x:13358338.89, y:3503549.84
------以下是Web墨卡托转WGS84------
经度:120.00, 纬度30.00


# 实验2
## 批处理下载华东师大闵行校区的地图切片数据（Esri卫星影像等切片服务数据），并生成world文件
比例尺级别为15，范围为（Web墨卡托投影坐标）：Xmin=13518000，Xmax=13521000，Ymin=3636000，Ymax=3638100

In [3]:
from math import *
import requests
import os

#地图范围
Xmin = 13518000
Xmax = 13521000
Ymin = 3636000
Ymax = 3638100
#比例尺级别 z
z = 15
#第z级的像元尺寸pixelLen
pixelLen = 40075014 / 2 ** 15
#切片的数量 colCount*rowCount
colCount = ceil((Xmax - Xmin) / pixelLen)
rowCount = ceil((Ymax - Ymin) / pixelLen)

#判断是否存在用于存放切片数据的文件夹,如果不存在则创建文件夹
folder = "./images"
if not os.path.exists(folder):
    os.makedirs(folder)

for i in range(colCount):
    for j in range(rowCount):
        #下载切片数据
        row = int((20037507 - (Ymin + j * pixelLen)) / pixelLen)
        col = int((Xmin + i * pixelLen - (-20037507)) / pixelLen)
        url = f"https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{row}/{col}.png"
        response = requests.get(url)
        f = open(f"./images/{z}_{row}_{col}.png", "wb")
        f.write(response.content)
        f.close()
        #生成world文件
        LU_y = 20037507 - (pixelLen * row)
        LU_x = (pixelLen * col) - 20037507
        x_resolution = pixelLen / 256
        y_resolution = pixelLen / 256
        text = f"{x_resolution}\n0\n0\n-{y_resolution}\n{LU_x}\n{LU_y}"
        f = open(f"./images/{z}_{row}_{col}.pngw", "w")
        f.write(text)
        f.close()